In [ ]:
!pip install opencv-python matplotlib
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install mediapipe
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
from google.colab import drive
import torch
import torchvision
import sys
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import mediapipe as mp
import gc
from segment_anything import sam_model_registry, SamPredictor
from keras.models import load_model
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
from datetime import datetime, timedelta
import pytz

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
 
folder_path = "/content/drive/My Drive/frames2/"
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)
number = 0
while (os.path.exists(folder_path) == False):
      drive.mount("/content/drive", force_remount=True)
for filename in os.listdir(folder_path):
    error_occurred = False
    if filename.endswith(".jpg"):
      with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        image = cv2.imread(folder_path + filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_height, image_width, _ = image.shape
        results = hands.process(image)
        annotated_image = image.copy()
        try: 
          for hand_landmarks in results.multi_hand_landmarks:
              landmark_coords = [(int(lm.x * image_width), int(lm.y * image_height)) for lm in hand_landmarks.landmark]
        except:
           error_occurred = True
      if (error_occurred == False):
        predictor.set_image(image)
        input_point = np.array(landmark_coords)
        input_label = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
        masks, _, _ = predictor.predict(point_coords=input_point, point_labels=input_label, multimask_output=False,)
        white_image = np.full_like(image, 255)
        white_image[masks[0] == 0] = [0, 0, 0]
        out = np.zeros_like(image) 
        out[white_image == 255] = image[white_image == 255]
        mpimg.imsave("/content/drive/My Drive/done2/" + str(number) + '.jpg', out)
        fig, ax = plt.subplots(figsize=(10,10))
        ax.imshow(out)
        ax.axis('off')
        plt.show()
        number = number + 1
        gc.collect()
      os.remove(folder_path + filename)
try:
  drive.mount('/content/drive')
  model = load_model('/content/drive/My Drive/Build/converted_keras2/keras_model.h5')
  labels = open('/content/drive/My Drive/Build/converted_keras2/labels.txt', 'r').readlines()
  weights = [0.1, 0.2, 0.3, 0.4] # Example weights, adjust as necessary
  folder_path = '/content/drive/My Drive/done2/' # Replace with path to your folder of photos
  probabilities_sum = [0] * len(labels)
  frame_count = 0
  for file_name in os.listdir(folder_path):
      if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
          image_path = os.path.join(folder_path, file_name)
          image = cv2.imread(image_path)
          hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
          image = cv2.resize(hsv, (224, 224), interpolation=cv2.INTER_AREA)
          image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
          image = (image / 127.5) - 1
          probabilities = model.predict(image)[0]
          for i, prob in enumerate(probabilities):
              probabilities_sum[i] += weights[min(int(prob * 10), 3)] # Apply weights based on the certainty of the prediction
          frame_count += 1

  avg_probabilities = [prob / frame_count for prob in probabilities_sum]
  category = labels[np.argmax(avg_probabilities)]
except:
  category = "Noah B. Schwartz"
gc = gspread.authorize(creds)
spreadsheet_title = 'SantiScan Data Base'
sh = gc.open(spreadsheet_title)
worksheet = sh.sheet1
data = [category]
last_row = len(worksheet.get_all_values()) + 1
mt_tz = pytz.timezone('US/Mountain')
current_date_time = datetime.now(mt_tz) - timedelta(minutes=2)
current_date = current_date_time.strftime('%m-%d-%Y')
current_time = current_date_time.strftime('%I:%M %p')
worksheet.append_row([data[0], current_date, current_time], value_input_option='USER_ENTERED')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-mhgb0pk2
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-mhgb0pk2
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 567662b0fd33ca4b022d94d3b8de896628cd32dd
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36610 sha256=13dff47c3a6b93974e3f057c24dc8ace2c2e0a0fff952758e531a6b83aee1542
  Stored in directory: /tmp/pip-ephem-wheel-cache-3smuof15/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 29.3 MB/s eta 0